<h2 style="color:red">
    <p>
        MÔN BIG DATA 2019
    </p>
    <p>
        Nhóm 13 Le Nguyen Son Nguyen - CH1702039
    </p>
</h2>

<h3 style="color:red">
    <p>
        SỬ DỤNG SPARK VÀ SPARK MACHINE LEARNING LIBRARY ĐỂ TRAIN MODEL PHÂN TÍCH NGỮ NGHĨA DÙNG MULTILAYER PERCEPTRONS (BASIC DEEP LEARNING) có sẵn trong Spark MLib
    </p>    
</h3>
<p>
    <a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#multilayer-perceptron-classifier">https://spark.apache.org/docs/latest/ml-classification-regression.html#multilayer-perceptron-classifier</a>
    <img src="https://www.researchgate.net/profile/Andreas_Holzinger/publication/320687279/figure/fig7/AS:561595223691264@1510906143097/Deep-feed-forward-neural-network-with-two-hidden-layers-blue-balls-In-addition-the.png" height="40%" width="40%" />
    
</p>

<h4>
    <p style="color:blue">
        Khai báo các biến môi trường để Spark sử dụng trong quá trình chạy
    </p>
    </h4>

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/spark/spark-2.4.3-bin-hadoop2.7"

<h4>
    <p style="color: blue">
        Dùng gói findSpark để khởi tạo môi trường chạy
    </p>
</h4>

In [2]:
import findspark
findspark.init()

<h4>
    <p style="color:blue">
        Import các thư viện của PySpark
    </p>
</h4>

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import MultilayerPerceptronClassifier
from os import listdir

<h4>
    <p style="color:blue">
        Khai báo số memory mà mỗi Worker sẽ sử dụng để thực thi task
    </p>
</h4>

In [4]:
SparkContext.setSystemProperty('spark.executor.memory', '6656m')

<h4>
    <p>
        Khai báo địa chỉ của Spark master, lưu ý là phải thay đổi địa chỉ cho phù hợp với từng Spark cluster
    </p>
</h4>

In [5]:
# tạo spark context, lưu ý nhớ thay đổi lại địa chỉ của spark cho phù hợp
sc = SparkContext("spark://10.255.255.6:7077", "Sematic Analysic training data")
spark = SparkSession.builder.getOrCreate()

<h4>
    <p style="color:blue">
        In ra đối tượng Spark Context
    </p>
</h4>

In [6]:
sc

<SparkContext master=spark://10.255.255.6:7077 appName=Sematic Analysic training data>

<h4>
    <p style="color:blue">
        In ra đối tượng Spark Session
    </p>
</h4>

In [7]:
spark

In [8]:
# khai báo một số hằng số
NUMBER_INSTANCE_FOR_PC = 12500
NUMBER_FEATURE_VECTOR = 500

<h4>
    <p style="color:blue">
        Dữ liệu có 2 tập:
    </p>
    <p style="color:red">
        Positive: nhận xét tích cực, chứa các câu nhận xét mang ý nghĩa tích cực, sẽ được đánh label là 1
    </p>
    <p style="color:red">
        Negative: nhận xét tiêu cực, chứa các câu nhận xét mang ý nghĩa tiêu cực, sẽ được đánh label là 0
    </p>
</h4>

In [9]:
# đường dẫn đến tập train positive
TRAIN_PATH_POS = "./semantic_data/aclImdb/train/pos/"
# đường dẫn đến tập train negative
TRAIN_PATH_NEG = "./semantic_data/aclImdb/train/neg/"

In [10]:
# biến lưu dữ liệu train cả positive và negative
data_train = []

In [11]:
# đọc dữ liệu của data train positive
listfile_pos = listdir(TRAIN_PATH_POS)
listfile_pos = enumerate(listfile_pos)
listfile_pos

<h4>
    <p style="color:blue">
        Đọc và xử lý để lấy dữ liệu train (positive)
    </p>
</h4>

In [12]:
counter = 1
for index, file_pos in listfile_pos:
    try:
        with open(TRAIN_PATH_POS + file_pos, 'r') as f_pos:
            noidung = f_pos.read()
            noidung = noidung.lower()
            # đánh label cho tập data train positive là 1
            data_train.append((index, noidung, 1))
        if counter >= NUMBER_INSTANCE_FOR_PC:
            break
        else:
            counter = counter + 1
    except Exception as e:
        print(e)

<h4>
    <p style="color:blue">
        In ra dữ liệu Train (positive)
    </p>
</h4>

In [13]:
len(data_train)

12500

In [14]:
# lấy dữ liệu cho tập data train negative
listfile_neg = listdir(TRAIN_PATH_NEG)
listfile_neg = enumerate(listfile_neg)
listfile_neg

<h4>
    <p style="color:blue">
        Đọc và xử lý để lấy dữ liệu train (negative)
    </p>
</h4>

In [15]:
counter = 1
for index, file_neg in listfile_neg:
    try:
        with open(TRAIN_PATH_NEG + file_neg) as f_neg:
            noidung_neg = f_neg.read()
            noidung_neg = noidung_neg.lower()
            # đánh label cho data train negative là 0
            data_train.append((index, noidung_neg, 0))
            if counter >= NUMBER_INSTANCE_FOR_PC:
                break
            else:
                counter = counter + 1
    except Exception as e:
        print(e)

<h4>
    <p>
        Tổng số dòng dữ liệu training là 25,000 dòng
    </p>
</h4>

In [16]:
len(data_train)

25000

<h4>
    <p style="color:red">
        Tạo đối tượng DataFrame trong Spark để chứa dữ liệu train, đây là kiểu dữ liệu chuẩn trong việc xử lý phân tán và có thể thao tác dưới dạng SQL, lúc này dữ liệu train đã ở dạng phân tán khắp cluster
    </p>
</h4>

In [17]:
# tạo DataFrame trong Spark
sentenceDataFrame_train = spark.createDataFrame(data_train, ["id", "sentence", "label"])

In [18]:
sentenceDataFrame_train.show(50)

+---+--------------------+-----+
| id|            sentence|label|
+---+--------------------+-----+
|  0|the sopranos was ...|    1|
|  1|revenge is the th...|    1|
|  2|i know some peopl...|    1|
|  3|my very favorite ...|    1|
|  4|i don't understan...|    1|
|  5|my name is heathe...|    1|
|  6|when the bourne i...|    1|
|  7|this was one of t...|    1|
|  8|the life and time...|    1|
|  9|watch on the rhin...|    1|
| 10|skip mccoy (richa...|    1|
| 11|though i really d...|    1|
| 12|unfortunately, be...|    1|
| 13|i have seen dirty...|    1|
| 14|a wonderful movie...|    1|
| 15|this film has sce...|    1|
| 16|to me, this revie...|    1|
| 17|when you typicall...|    1|
| 18|im not a big tim ...|    1|
| 19|knowing when to e...|    1|
| 20|getting to work o...|    1|
| 21|i think "anyway.....|    1|
| 22|i have waited a l...|    1|
| 23|i loved this film...|    1|
| 24|pet sematary is a...|    1|
| 25|as a sequel,this ...|    1|
| 26|i think that most...|    1|
| 27|visco

<h4>
    <p style="color:blue">
        Tạo một cấu trúc Pipeline trong Spark gồm có:
    </p>
    <p style="color:red">
        1. Tokenizer để rời rạc hóa câu trong data train. Ví dụ: "I am bad" --> [ "I", "am", "bad" ]
    </p>
    <p style="color:red">
        2. StopWordsRemover để loại bỏ các từ không mang giá trị phân biệt, ví dụ: I, You,...
    </p>
    <p style="color:red">
        3. HashingTF để tính TF (Tern Frequency)
    </p>
    <p style="color:red">
        4. IDF để tính IDF, lúc này ứng với mỗi câu sẽ có một cột features là một vector đại diện cho dạng biểu diễn của câu đó
    </p>
    <p style="color:red">
        5. MultilayerPerceptronClassifier là model Deep Learning trong Spark dùng để train tập dữ liệu đã chuẩn bị ở trên
    </p>
</h4>

In [19]:
# Tokenize (rời rạc hóa) word trong câu in data train
tk_train = Tokenizer(inputCol="sentence", outputCol="words_raw")

# loại bỏ từ stop word ---> là những từ không mang ý nghĩa phân biệt nhiều,
# cũng như lập lại quá nhiều lần trong các câu, ví dụ: I, You,...
stop_remover_train = StopWordsRemover(inputCol="words_raw", outputCol="filtered")

# tính TF (Tern Frequency)
tf_train = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=NUMBER_FEATURE_VECTOR)

# tính IDF (Invert Tern Frequency)
idf_train = IDF(inputCol="rawFeatures", outputCol="features")

# sử dụng một mô hình classification là MultiLayer, định nghĩa số Layers
layers = [NUMBER_FEATURE_VECTOR, NUMBER_FEATURE_VECTOR * 2, NUMBER_FEATURE_VECTOR, 2]
mlp_trainer = MultilayerPerceptronClassifier(featuresCol="features", labelCol="label", layers=layers, seed=123456)

In [20]:
tk_train

Tokenizer_2582e78e57d0

In [21]:
stop_remover_train

StopWordsRemover_dd87be6c96ed

In [22]:
tf_train

HashingTF_9b87097d3695

In [23]:
idf_train

IDF_65ddc58ad95a

In [24]:
mlp_trainer

MultilayerPerceptronClassifier_c986d7557bfa

<h4>
    <p style="color:red">
        Tạo đối tượng Pipeline gồm các bước như đã định nghĩa ở trên
    </p>
</h4>

In [25]:
pl = Pipeline(stages=[tk_train, stop_remover_train, tf_train, idf_train, mlp_trainer])

<h4>
    <p>
        Bắt đầu Train tập dữ liệu
    </p>
</h4>

In [26]:
model = pl.fit(sentenceDataFrame_train)

In [27]:
transformedFeatures = model.transform(sentenceDataFrame_train)
transformedFeatures

DataFrame[id: bigint, sentence: string, label: bigint, words_raw: array<string>, filtered: array<string>, rawFeatures: vector, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [28]:
transformedFeatures.show(50)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|            sentence|label|           words_raw|            filtered|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0|the sopranos was ...|    1|[the, sopranos, w...|[sopranos, probab...|(500,[5,12,19,21,...|(500,[5,12,19,21,...|[-1.1170839244941...|[0.20644935629981...|       1.0|
|  1|revenge is the th...|    1|[revenge, is, the...|[revenge, theme, ...|(500,[10,18,26,47...|(500,[10,18,26,47...|[-1.3511970707784...|[0.14431969220346...|       1.0|
|  2|i know some peopl...|    1|[i, know, some, p...|[know, people, th...|(500,[0,1,3,9,12,...|(500,[0,1,3,9,12,...|[-2.1759199481953...|[0.0302157183

<h4>
    <p style="color:red">
        Lưu lại model đã train, Lưu ý: là phải lưu model dưới dạng một hệ thống file phân tán như Hadoop
    </p>
</h4>

In [29]:
model.write().overwrite().save("hdfs://s14e18f4e58324b66b78ecd8c831a0c6413-master.uitlab.com:9000/sparkdata/pl2.model")

<h4>
    <p style="color:blue">
        Khai cần sử dụng lại model đã lưu thì dùng lệnh dưới đây
    </p>
</h4>

In [30]:
from pyspark.ml import PipelineModel

In [31]:
pl_model = PipelineModel.load("hdfs://s14e18f4e58324b66b78ecd8c831a0c6413-master.uitlab.com:9000/sparkdata/pl2.model")

<h4>
    <p>
        Kết Thúc một phiên làm việc trong Spark
    </p>
</h4>

In [32]:
print("end_of_train_and_save_Multilayers(Basic Deep Model) in Spark")
spark.stop()

end_of_train_and_save_Multilayers(Basic Deep Model) in Spark
